In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# load data
pcd_file = "/Users/matias/digiforest/anymal_clouds/forest_cloud.pcd"
pcd = o3d.t.io.read_point_cloud(pcd_file)
print(pcd)
o3d.visualization.draw_geometries([pcd.to_legacy()],
                                  zoom=0.5,
                                  front=[0.79, 0.02, 0.60],
                                  lookat=[2.61, 2.04, 1.53],
                                  up=[-0.60, -0.012, 0.79])

In [ ]:
# Visualize segmentation results
def visualize(cloud, labels):
    cmap = plt.get_cmap("tab20")
    colors = cmap(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([cloud],
                                    zoom=0.5,
                                    front=[0.79, 0.02, 0.60],
                                    lookat=[2.61, 2.04, 1.53],
                                    up=[-0.60, -0.012, 0.79])

In [ ]:
mask = (pcd.point.normals[:, 2]>=-0.5) & (pcd.point.normals[:, 2]<=0.5)
pcd = pcd.select_by_mask(mask).to_legacy()

In [ ]:
pcd = pcd.voxel_down_sample(voxel_size=0.5)
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.5,
                                  front=[0.79, 0.02, 0.60],
                                  lookat=[2.61, 2.04, 1.53],
                                  up=[-0.60, -0.012, 0.79])

In [ ]:
# DBSCAN (open3d)
eps = 0.5
min_cluster_size = 50
labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_cluster_size, print_progress=False))
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# DBSCAN (sklearn)
from sklearn.cluster import DBSCAN
X = np.asarray(pcd.points)[:,:2]
eps = 0.2
min_cluster_size = 20
db = DBSCAN(eps=0.3, min_samples=min_cluster_size).fit(X)
labels = db.labels_
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# HDBSCAN
from sklearn.cluster import HDBSCAN
X = np.asarray(pcd.points)[:,:2]
min_cluster_size = 100
hdb = HDBSCAN(min_cluster_size=20)
hdb.fit(X)
labels = hdb.labels_
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# OPTICS (O(n^2)- Not Suitable for large pointclouds)
from sklearn.cluster import OPTICS, cluster_optics_dbscan
X = np.asarray(pcd.points)[:,:2]
optics = OPTICS(min_samples=50, xi=0.05, max_eps=1.0, algorithm="kd_tree")
optics.fit(X)
labels = cluster_optics_dbscan(reachability=optics.reachability_,
                               core_distances=optics.core_distances_,
                               ordering=optics.ordering_,
                               eps=0.5)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# KMeans
from sklearn.cluster import KMeans
num_clusters = 100

X = np.asarray(pcd.points)[:,:2]
labels = KMeans(n_clusters=num_clusters, n_init= "auto").fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# Bisecting KMeans
from sklearn.cluster import BisectingKMeans
num_clusters = 100

X = np.asarray(pcd.points)[:,:2]
labels = BisectingKMeans(n_clusters=num_clusters).fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# MeanShift
from sklearn.cluster import MeanShift

X = np.asarray(pcd.points)[:,:2]
labels = MeanShift().fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# GaussianMixture
from sklearn.mixture import GaussianMixture

X = np.asarray(pcd.points)[:,:2]
labels = GaussianMixture(n_components=100).fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# AgglomerativeClustering
from sklearn.cluster import AgglomerativeClustering

X = np.asarray(pcd.points)[:,:2]
labels = AgglomerativeClustering(n_clusters=100).fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# SpectralClustering
from sklearn.cluster import SpectralClustering

X = np.asarray(pcd.points)[:,:2]
labels = SpectralClustering(n_clusters=100).fit_predict(X)
max_label = labels.max()
print("num_clusters = %d" % (max_label + 1))
visualize(pcd, labels)

In [ ]:
# Euclidean clustering
from sklearn.neighbors import KDTree
X = np.asarray(pcd.points)
ec = EuclideanClustering(X, distance_threshold = 1.0, min_cluster_size=100, max_cluster_size=10000)
visualize(pcd, labels)

In [ ]:
#Euclidean Clustering
import pcl
import numpy as np

pcd_file = "/home/nived/logs/digiforest_analysis_test_data/anymal_data/payload_clouds_no_ground/cloud_1683207437_293473482.pcd"
cloud = pcl.load(pcd_file)

#   // Create the filtering object: downsample the dataset using a leaf size of 5cm
vg = cloud.make_voxel_grid_filter()
vg.set_leaf_size(0.05, 0.05, 0.05)
cloud_filtered = vg.filter()

# Creating the KdTree object for the search method of the extraction
tree = cloud_filtered.make_kdtree()
# tree = cloud_filtered.make_kdtree_flann()

# std::vector<pcl::PointIndices> cluster_indices;
ec = cloud_filtered.make_EuclideanClusterExtraction()
ec.set_ClusterTolerance(0.10)
ec.set_MinClusterSize(100)
ec.set_MaxClusterSize(25000)
ec.set_SearchMethod(tree)
cluster_indices = ec.Extract()
print('cluster_indices : ' + str(len(cluster_indices)) + " count.")

In [ ]:
# Get valid clusters


In [ ]:
# Visualize and write clusters to disk
cloud_cluster = pcl.PointCloud_PointXYZRGB()

for j, indices in enumerate(cluster_indices):
    # cloudsize = indices
    print('indices = ' + str(len(indices)))
    # cloudsize = len(indices)
    points = np.zeros((len(indices), 4), dtype=np.float32)
    # points = np.zeros((cloudsize, 3), dtype=np.float32)

    # for indice in range(len(indices)):
    for i, indice in enumerate(indices):
        # print('dataNum = ' + str(i) + ', data point[x y z]: ' + str(cloud_filtered[indice][0]) + ' ' + str(cloud_filtered[indice][1]) + ' ' + str(cloud_filtered[indice][2]))
        # print('PointCloud representing the Cluster: ' + str(cloud_cluster.size) + " data points.")
        points[i][0] = cloud_filtered[indice][0]
        points[i][1] = cloud_filtered[indice][1]
        points[i][2] = cloud_filtered[indice][2]
        points[i][3] = 255 << 16 | 255 << 8 | 255

    cloud_cluster.from_array(points)
    ss = "cloud_cluster_" + str(j) + ".pcd"
    pcl.save(cloud_cluster, ss)